In [1]:
import glob
import os
import pandas as pd

l = glob.glob("./path/*.jpg")
img_path = []
for i in l:
    bn = os.path.basename(i)
    img_path.append(bn)

df_img = pd.DataFrame(img_path, columns = ["img_path"])
df = pd.read_csv("data.csv")
df_r = df[df["img_path"].isin(df_img["img_path"])]
print(len(df), len(df_img), len(df_r))

97230 198753 91565


In [13]:
df_r.to_csv("data.csv", index=False)

In [1]:
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import datetime

def darken_blend2(img1, img2, img3):

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    img3_gray = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)

    min_gray = cv2.min(cv2.min(img1_gray, img2_gray), img3_gray)
    mask1 = (min_gray == img1_gray) * 1.0
    mask2 = (min_gray == img2_gray) * 1.0
    mask2 = cv2.min(np.zeros((200, 200, 3)), mask2 - mask1)
    mask3 = (min_gray == img3_gray) * 1.0
    mask3 = cv2.min(np.zeros((200, 200, 3)), mask3 - mask1)
    mask3 = cv2.min(np.zeros((200, 200, 3)), mask3 - mask2)

    ret_img = np.zeros((200, 200, 3))
    ret_img[:,:,0] = (mask1 * img1[:,:,0]) + mask2 * img2[:,:,0] + mask3 * img3[:,:,0]
    ret_img[:,:,1] = (mask1 * img1[:,:,1]) + mask2 * img2[:,:,1] + mask3 * img3[:,:,1]
    ret_img[:,:,2] = (mask1 * img1[:,:,2]) + mask2 * img2[:,:,2] + mask3 * img3[:,:,2]
    return ret_img

def darken_blend(img1, img2, img3):
    '''
    3枚のcv2の画像を用いて暗合成を行う関数
    '''
    dark_blend = cv2.min(cv2.min(img1, img2), img3)
    return dark_blend


def dark_diff(target_img, refernence_img1, refernence_img2):
    '''
    暗合成差分法を行う関数
    '''
    dark_blend_img = darken_blend(target_img, refernence_img1, refernence_img2)
    return target_img - dark_blend_img

#def otsu_threshold(img):
#    '''
#    Otsuの2値化を行う関数
#    '''
#    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#    _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#    return thresh
#
#
#def otsu_diff(target_img, reference_img):
#    '''
#    Otsu差分法を行う関数
#    '''
#    target_bi = otsu_threshold(target_img)
#    reference_bi = otsu_threshold(reference_img)
#    mask1 = (target_bi == 255) * 1
#    mask2 = ((target_bi == 255) & (reference_bi == 255)) * 1
#    mask = mask1 - mask2
#    target_img_ = target_img.copy()
#    target_img_[:, :, 0] = target_img_[:, :, 0] * mask
#    target_img_[:, :, 1] = target_img_[:, :, 1] * mask
#    target_img_[:, :, 2] = target_img_[:, :, 2] * mask
#    return target_img_

In [2]:
month = "202402"

df = pd.read_csv("all/{}.csv".format(month)).reset_index(drop=True)
# df = pd.read_csv("L:/akeyama/202311.csv")

img_path = df["img_path"]
img_path_no_jpgs = []

for row in tqdm(img_path, total = len(img_path)):
    img_path_no_jpgs.append(row[:-4])
df['datetime'] = img_path_no_jpgs
df['datetime'] = pd.to_datetime(df['datetime'])

100%|██████████| 17292/17292 [00:00<00:00, 2467606.99it/s]


In [3]:
def image_process(img_dir_path, target_img_basename, reference_img_basename1, reference_img_basename2):

    target_img = cv2.imread(os.path.join(img_dir_path, target_img_basename))
    reference_img1 = cv2.imread(os.path.join(img_dir_path, reference_img_basename1))
    reference_img2 = cv2.imread(os.path.join(img_dir_path, reference_img_basename2))
    # csvにあってimage_dirに存在しなかったらNoneを返す
    if (target_img is None) | (reference_img1 is None) | (reference_img2 is None):
        return None
    dark_blend_diff_output = dark_diff(target_img, reference_img1, reference_img2)
    return dark_blend_diff_output

In [5]:
month = '202402'

In [6]:
complete_img_paths = []

datetimes = df['datetime']
img_paths = df['img_path']

img_dir_path_800 = "image/103_resize/trim_800/{}".format(month)
dest_dir_path_dark_800 = "image/104_dark_blend_diff/{}".format(month)

os.makedirs(dest_dir_path_dark_800, exist_ok = True)

for i in tqdm(range(len(datetimes))):
    if i < 2:
        continue
    target_img_basename = img_paths[i]
    reference_img_basename1 = img_paths[i-1]
    reference_img_basename2 = img_paths[i-2]
    
    target_datetime = datetimes[i]
    reference_datetime1 = datetimes[i-1]
    reference_datetime2 = datetimes[i-2]
    if target_datetime + datetime.timedelta(seconds=-10) != reference_datetime1:
        continue
    if reference_datetime1 + datetime.timedelta(seconds=-10) != reference_datetime2:
        continue

    dark_blend_diff_800 = image_process(
        img_dir_path_800, target_img_basename, reference_img_basename1, reference_img_basename2
    )

    if dark_blend_diff_800 is None:
        continue

    cv2.imwrite(
        os.path.join(dest_dir_path_dark_800, target_img_basename),
        dark_blend_diff_800
    )
    complete_img_paths.append(img_paths[i])

complete_df = df[df['img_path'].isin(complete_img_paths)]
complete_df.drop(["datetime"], axis = 1, inplace=True)
complete_df.to_csv("{}_dark.csv".format(dest_dir_path_dark_800))

100%|██████████| 17292/17292 [02:11<00:00, 131.72it/s]
C:\Users\water\AppData\Local\Temp\ipykernel_8960\838161319.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete_df.drop(["datetime"], axis = 1, inplace=True)


In [5]:
flow = complete_df

In [6]:
complete_img_paths = []
month = "202307"

datetimes = df['datetime']
img_paths = df['img_path']

img_dir_path_800 = "image/103_resize/trim_800/{}".format(month)
dest_dir_path_dark_800 = "image/104_dark_blend_diff/{}".format(month)

# os.makedirs(dest_dir_path_dark_800, exist_ok = True)

for i in tqdm(range(len(datetimes))):
    if i < 2:
        continue
    target_img_basename = img_paths[i]
    reference_img_basename1 = img_paths[i-1]
    reference_img_basename2 = img_paths[i-2]
    
    target_datetime = datetimes[i]
    reference_datetime1 = datetimes[i-1]
    reference_datetime2 = datetimes[i-2]
    if target_datetime + datetime.timedelta(seconds=-10) != reference_datetime1:
        continue
    if reference_datetime1 + datetime.timedelta(seconds=-10) != reference_datetime2:
        continue

    dark_blend_diff_800 = image_process(
        img_dir_path_800, target_img_basename, reference_img_basename1, reference_img_basename2
    )

    if dark_blend_diff_800 is None:
        continue

    cv2.imwrite(
        os.path.join(dest_dir_path_dark_800, target_img_basename),
        dark_blend_diff_800
    )
    complete_img_paths.append(img_paths[i])

complete_df = df[df['img_path'].isin(complete_img_paths)]
complete_df.drop(["datetime"], axis = 1)
# complete_df.to_csv("{}.csv".format(dest_dir_path_dark_800))

100%|██████████| 62952/62952 [05:08<00:00, 204.27it/s] 


,Unnamed: 0,img_path,raw_pH,treated_pH,camera_pH,raw_t,treated_t,tem,Al
28934,28934,20230701030020.jpg,7.22,6.95,7.46,15.810,2.032,23.440,1.0
28935,28935,20230701030030.jpg,7.22,6.95,7.46,15.810,2.032,23.440,1.0
28936,28936,20230701030040.jpg,7.22,6.95,7.46,15.810,2.032,23.440,1.0
28937,28937,20230701030050.jpg,7.22,6.95,7.46,15.810,2.032,23.440,1.0
28938,28938,20230701030100.jpg,7.21,6.95,7.46,15.740,1.957,23.390,1.0
...,...,...,...,...,...,...,...,...,...
57899,57899,20230712173150.jpg,7.83,7.47,7.59,9.662,3.356,29.223,0.5
57900,57900,20230712173200.jpg,7.83,7.47,7.59,9.775,3.493,29.324,0.5
57901,57901,20230712173210.jpg,7.83,7.47,7.59,9.775,3.493,29.324,0.5
57902,57902,20230712173220.jpg,7.83,7.47,7.59,9.775,3.493,29.324,0.5


In [7]:
flowdown = pd.concat([flow, complete_df])
flowdown = flowdown.sort_values("img_path").reset_index(drop=True)
flowdown.to_csv("{}_flowdown.csv".format(dest_dir_path_dark_800))

In [7]:
import pandas as pd
import glob
import os

l = glob.glob("C:/Users/water/image/104_dark_blend_diff/20230/*.jpg")
bn = []
for i in l:
    basename = os.path.basename(i)
    bn.append(basename)

df_bn = pd.DataFrame(bn, columns=["img_path"])
df = pd.read_csv("all/all_202308.csv")

df1 = df[df["img_path"].isin(df_bn["img_path"])]

In [6]:
df1.to_csv("all/all_202309.csv", index=False)